In [1]:
import os
import glob
import subprocess
import numpy as np
import pandas as pd
import pyvista as pv
import shapeworks as sw
import matplotlib.pyplot as plt

from pathlib import Path
from scipy.spatial import cKDTree

In [4]:
# Config
muscle_name = 'semimembranosus'  # Change this to the desired muscle name
side = 'left'
modifiers = ['rigid', 'novolume']

# Define directories
data_dir = "data/"
muscle_dir = f"{data_dir}{muscle_name}/"
bin_dir = f"{muscle_dir}bin/"
base_dir = "projects/"
project_dir = f"{base_dir}{muscle_name}/"

os.makedirs(base_dir, exist_ok=True)
os.makedirs(project_dir, exist_ok=True)

for modifier in modifiers:
    mesh_dir = f"{muscle_dir}mesh_{modifier}/{side}/"
    spreadsheet_file = f"{project_dir}{muscle_name}_{modifier}_{side}.swproj"
    output_dir = Path(f"{project_dir}{muscle_name}_{modifier}_{side}_analysis")
    output_dir.mkdir(exist_ok=True)


In [3]:
import pandas as pd

# Load metadata from Excel
meta = pd.read_excel("MOTION_characteristics.xlsx")

# Normalize columns to lower-case
meta.columns = [c.lower() for c in meta.columns]

# Determine correct ID column name
id_col = [c for c in meta.columns if "participant" in c and "id" in c][0]  # auto-detect
sex_col = [c for c in meta.columns if "sex" in c][0]

# Build ID → sex mapping
# MRI_Sex: 1 = male, 2 = female
id_to_sex = {}
for _, row in meta.iterrows():
    pid = str(row[id_col]).strip()
    sex_num = row[sex_col]
    if sex_num == 1:
        id_to_sex[pid] = "male"
    elif sex_num == 2:
        id_to_sex[pid] = "female"


# Optimize

In [ ]:
# Get filenames
mesh_extension = '.ply'

for modifier in modifiers:
    print(f"\n=== Processing modifier: {modifier} ===")
    mesh_dir = f"{muscle_dir}mesh_{modifier}/{side}/"
    mesh_filenames = sorted(glob.glob(mesh_dir + '*' + mesh_extension))
    print(f" Total number of distance transform files: {len(mesh_filenames)}")

    # Get shape names from binary filenames
    shape_names = [os.path.splitext(f.split('/')[-1])[0] for f in mesh_filenames]
    print(" Shape names:", shape_names)

    # Parameters
    use_single_scale = False  # Set to True for single scale optimization

    group_name = ["sex"]     # Only grouping by sex
    group_values = ["male", "female"]

    subjects = []
    number_domains = 1

    def extract_id(shape_name):
        # Split at the first underscore
        base = shape_name.split("_")[0]
        return base  # e.g., "MOTION001"


    for i in range(len(shape_names)):
        shape = shape_names[i]
        pid = extract_id(shape)

        if pid not in id_to_sex:
            raise ValueError(f"PID {pid} from mesh {shape} not found in Excel!")

        sex_group = id_to_sex[pid]   # "male" or "female"

        subject = sw.Subject()
        subject.set_number_of_domains(number_domains)
        subject.set_groomed_filenames(sw.utils.get_relative_paths([mesh_filenames[i]], project_dir))
        subject.set_group_values({group_name[0]: sex_group})

        subjects.append(subject)
        print(f" ✅ Added subject {i+1}: {shape}  →  {sex_group}")

    # Set up the project
    project = sw.Project()
    project.set_subjects(subjects)
    parameters = sw.Parameters()

    # Optimization Parameters
    parameter_dictionary = {
        "number_of_particles": 256,
        "use_normals": 1,
        "normals_strength": 10.0,
        "checkpointing_interval": 200,
        "keep_checkpoints": 0,
        "iterations_per_split": 1000,
        "optimization_iterations": 2000,
        "starting_regularization": 200,
        "ending_regularization": 1,
        "initial_relative_weighting": 0.3,
        "relative_weighting": 1.0,
        "procrustes_interval": 0,
        "procrustes_scaling": 0,
        "save_init_splits": 1,
        "verbosity": 0
    }

    if not use_single_scale:
        parameter_dictionary["multiscale"] = 1
        parameter_dictionary["multiscale_particles"] = 32

    for key in parameter_dictionary:
        parameters.set(key, sw.Variant([parameter_dictionary[key]]))
    project.set_parameters("optimize", parameters)

    # Save
    spreadsheet_file = f"{project_dir}{muscle_name}_{modifier}_{side}.swproj"
    project.save(spreadsheet_file)
    print(f" 💾 Project saved to {spreadsheet_file}")

    # Run optimization
    optimize_cmd = [
        'shapeworks',
        'optimize',
        '--progress',
        '--name',
        spreadsheet_file
    ]
    try:
        subprocess.check_call(optimize_cmd)
    except subprocess.CalledProcessError as e:
        print(f" Optimization for {modifier} failed with error:")
        print(e)
    


=== Processing modifier: rigid ===
 Total number of distance transform files: 101
 Shape names: ['MOTION001_left', 'MOTION002_left', 'MOTION003_left', 'MOTION004_left', 'MOTION005_left', 'MOTION006_left', 'MOTION007_left', 'MOTION008_left', 'MOTION009_left', 'MOTION010_left', 'MOTION011_left', 'MOTION012_left', 'MOTION013_left', 'MOTION014_left', 'MOTION015_left', 'MOTION016_left', 'MOTION017_left', 'MOTION018_left', 'MOTION019_left', 'MOTION020_left', 'MOTION021_left', 'MOTION022_left', 'MOTION023_left', 'MOTION024_left', 'MOTION025_left', 'MOTION026_left', 'MOTION027_left', 'MOTION028_left', 'MOTION029_left', 'MOTION030_left', 'MOTION031_left', 'MOTION032_left', 'MOTION033_left', 'MOTION034_left', 'MOTION035_left', 'MOTION036_left', 'MOTION037_left', 'MOTION038_left', 'MOTION039_left', 'MOTION040_left', 'MOTION041_left', 'MOTION042_left', 'MOTION043_left', 'MOTION044_left', 'MOTION045_left', 'MOTION046_left', 'MOTION047_left', 'MOTION048_left', 'MOTION049_left', 'MOTION050_left', 'M

# Analyze

In [6]:
modifier = ["rigid", "novolume"][1] ## Change this to select which model's particles to analyze
spreadsheet_file = f"{project_dir}{muscle_name}_{modifier}_{side}.swproj"

analyze_cmd = ('ShapeWorksStudio ' + spreadsheet_file).split()
subprocess.check_call(analyze_cmd)

[2026-02-23 12:59:29.855] [info] ShapeWorks Studio 6.6.1 initializing...


libpng warning: iCCP: known incorrect sRGB profile


[2026-02-23 12:59:30.759] [info] ShapeWorks Studio Initialized
[2026-02-23 12:59:31.429] [info] Loading Project: projects/semimembranosus/semimembranosus_novolume_left.swproj
[2026-02-23 12:59:32.472] [info] Project loaded: projects/semimembranosus/semimembranosus_novolume_left.swproj
[2026-02-23 12:59:45.478] [info] Wrote: /Users/fadiahanifa/Documents/Akademik/Master/Major Project/Code Deliverables/projects/semimembranosus/semimembranosus_novolume_left_analysis/mean_mesh.vtk
[2026-02-23 12:59:47.619] [info] Log Closed


Shut Down Optimization Threads


0

## Calculate Stuff

In [8]:
import numpy as np
import pandas as pd
import re
import json
from pathlib import Path

# -------- CONFIG --------
modifier = ["rigid", "novolume"][1] ## Change this to select which model's particles to analyze
particles_dir = Path(f"{project_dir}{muscle_name}_{modifier}_{side}_particles")
output_dir = Path(f"{project_dir}{muscle_name}_{modifier}_{side}_analysis")
output_dir.mkdir(exist_ok=True)

char_path = "MOTION_characteristics.xlsx"


# 1. LOAD SEX TABLE (MOTIONXXX → male/female)
chars = pd.read_excel(char_path)
chars.columns = [c.strip().lower() for c in chars.columns]

id_col = [c for c in chars.columns if "participant" in c and "id" in c][0]
sex_col = [c for c in chars.columns if "sex" in c][0]

sex_map = {}
for _, row in chars.iterrows():
    pid = str(row[id_col]).strip()
    if row[sex_col] == 1:
        sex_map[pid] = "male"
    elif row[sex_col] == 2:
        sex_map[pid] = "female"

# 2. LOAD PARTICLES
def load_particles(pdir):
    files = sorted([f for f in pdir.glob("*local*.particles")])
    shapes = [np.loadtxt(f) for f in files]
    return shapes, files

shapes_all, files_all = load_particles(particles_dir)
print(f"🔍 Found {len(shapes_all)} total *_local.particles")

# Extract subject IDs
def extract_subject_id(path):
    fname = path.stem
    m = re.match(r"(MOTION\d+)", fname)
    return m.group(1) if m else fname

subject_ids = [extract_subject_id(f) for f in files_all]
sex_list = [sex_map.get(pid, None) for pid in subject_ids]

# 3️⃣ SUBSETS (ALL, M, F)

def subset_data(shapes, ids, sexes, group):
    if group == "all":
        idx = list(range(len(shapes)))
    else:
        idx = [i for i, s in enumerate(sexes) if s == group]

    shapes_sub = [shapes[i] for i in idx]
    ids_sub = [ids[i] for i in idx]
    return shapes_sub, ids_sub

subsets = {
    "all": subset_data(shapes_all, subject_ids, sex_list, "all"),
    "m": subset_data(shapes_all, subject_ids, sex_list, "male"),
    "f": subset_data(shapes_all, subject_ids, sex_list, "female"),
}

# ======================================================
# 4️⃣ PCA FUNCTION + STANDARDIZATION
# ======================================================
def run_pca_and_save(shapes, ids, tag="all"):
    if len(shapes) == 0:
        print(f"⚠️ Skipping {tag}: no samples.")
        return

    print(f"\n===== Running PCA for: {tag.upper()} ({len(shapes)} subjects) =====")

    # Build matrix
    data_matrix = np.stack([s.flatten() for s in shapes])
    mean_vector = np.mean(data_matrix, axis=0)
    mean_points = mean_vector.reshape(-1, 3)

    # Save mean shape
    if tag == "all":
        mean_path = output_dir / "mean.particles"
    else:
        mean_path = output_dir / f"mean_{tag}.particles"
    np.savetxt(mean_path, mean_points, fmt="%.8f")

    # PCA
    centered = data_matrix - mean_vector
    U, S, VT = np.linalg.svd(centered, full_matrices=False)
    eigenvalues = (S**2) / (len(data_matrix) - 1)

    # Save modes & eigenvalues
    if tag == "all":
        np.save(output_dir / "pca_modes.npy", VT)
        np.save(output_dir / "eigenvalues.npy", eigenvalues)
    else:
        np.save(output_dir / f"pca_modes_{tag}.npy", VT)
        np.save(output_dir / f"eigenvalues_{tag}.npy", eigenvalues)

    # Raw scores
    scores = centered @ VT.T

    # -----------------------
    # STANDARDIZATION
    # -----------------------
    score_means = np.mean(scores, axis=0)
    score_stds = np.std(scores, axis=0, ddof=1)

    scores_z = (scores - score_means) / score_stds

    n_modes = scores_z.shape[1]
    pc_cols = [f"PC{i+1}" for i in range(n_modes)]

    # Build dataframe (SubjectID + standardized scores)
    df = pd.DataFrame(scores_z, columns=pc_cols)
    df.insert(0, "SubjectID", ids)

    # Output filenames
    if tag == "all":
        scores_path = output_dir / "scores_all_standardized.csv"
        scaler_path = output_dir / "score_scaler_all.json"
    else:
        scores_path = output_dir / f"scores_{tag}_standardized.csv"
        scaler_path = output_dir / f"score_scaler_{tag}.json"

    # Save standardized scores
    df.to_csv(scores_path, index=False)
    print(f"✔ Saved standardized scores — {scores_path.name}")

    # Save scaler parameters
    scaler_info = {
        "score_means": score_means.tolist(),
        "score_stds": score_stds.tolist()
    }
    with open(scaler_path, "w") as f:
        json.dump(scaler_info, f, indent=4)

    print(f"✔ Saved scaler — {scaler_path.name}")

# ======================================================
# 5️⃣ RUN PCA (ALL / M / F)
# ======================================================
run_pca_and_save(*subsets["all"], tag="all")
run_pca_and_save(*subsets["m"], tag="m")
run_pca_and_save(*subsets["f"], tag="f")


🔍 Found 101 total *_local.particles

===== Running PCA for: ALL (101 subjects) =====
✔ Saved standardized scores — scores_all_standardized.csv
✔ Saved scaler — score_scaler_all.json

===== Running PCA for: M (46 subjects) =====
✔ Saved standardized scores — scores_m_standardized.csv
✔ Saved scaler — score_scaler_m.json

===== Running PCA for: F (55 subjects) =====
✔ Saved standardized scores — scores_f_standardized.csv
✔ Saved scaler — score_scaler_f.json


# Modes visualization

In [11]:
# ------------------------- CONFIG -------------------------
modifier = modifiers[1] ## Change this to select which model's particles to analyze
output_dir = Path(f"{project_dir}{muscle_name}_{modifier}_{side}_analysis")
particles_dir = Path(f"{project_dir}{muscle_name}_{modifier}_{side}_particles")

mean_particles_path = output_dir / "mean.particles"
pca_modes_path = output_dir / "pca_modes.npy"
eigenvalues_path = output_dir / "eigenvalues.npy"
scores_path = output_dir / "scores.csv"
mean_mesh_path = output_dir / "mean_mesh.vtk"

num_modes_to_visualize = 3
std_dev = 2  # for +/-2 std visualization
# ----------------------------------------------------------

In [12]:
import numpy as np
import pyvista as pv
from scipy.spatial import cKDTree
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib as mpl

# ----------------------------
# CONFIG
# ----------------------------
k_sd = std_dev                  # visualize mean + k_sd * SD
kNN = 5                         # 1 = nearest neighbor, 3–10 = smoother mapping
window_size = (1200, 1000)
cmap = "PuOr"               # red=positive(out), blue=negative(in)
output_dir = Path(f"{project_dir}{muscle_name}_{modifier}_{side}_analysis/pcs_heatmaps")
output_dir.mkdir(exist_ok=True)

# ----------------------------
# LOAD
# ----------------------------
mean_particles = np.loadtxt(mean_particles_path)         # (N, 3)
pca_modes = np.load(pca_modes_path)                      # (3N, M) or (M, 3N)
eigenvalues = np.load(eigenvalues_path)                  # (M,)
mean_mesh = pv.read(mean_mesh_path)

# ensure pca_modes is (3N, M)
if pca_modes.shape[0] != mean_particles.size:
    pca_modes = pca_modes.T
assert pca_modes.shape[0] == mean_particles.size, "Expected pca_modes to be (3N, M) after transpose."

tree = cKDTree(mean_particles)
mesh_pts = mean_mesh.points

def muscle_pca_axes(points):
    pts = np.asarray(points, dtype=float)
    pts = pts - pts.mean(axis=0, keepdims=True)
    _, _, vh = np.linalg.svd(pts, full_matrices=False)
    return vh  # rows: PC1, PC2, PC3

def get_muscle_camera_position(mesh, view="cross2", flip=False, dist_factor=7.5):
    axes = muscle_pca_axes(mesh.points)

    center = np.asarray(mesh.center, float)
    bounds = np.asarray(mesh.bounds, float).reshape(3, 2)
    radius = 0.5 * np.linalg.norm(bounds[:, 1] - bounds[:, 0])

    if view == "cross2":
        view_dir = axes[2]
        up = axes[0]
    elif view == "cross1":
        view_dir = axes[1]
        up = axes[0]
    elif view == "long":
        view_dir = axes[0]
        up = axes[2]
    else:
        raise ValueError(view)

    view_dir = np.asarray(view_dir, float)
    up = np.asarray(up, float)

    view_dir /= (np.linalg.norm(view_dir) + 1e-12)
    up /= (np.linalg.norm(up) + 1e-12)

    if flip:
        view_dir = -view_dir

    cam_pos = center + view_dir * radius * dist_factor
    return [cam_pos.tolist(), center.tolist(), up.tolist()]


def map_vectors_particles_to_mesh(vec_particles, k=1):
    """vec_particles: (N,3) -> returns (V,3)"""
    if k == 1:
        nn_idx = tree.query(mesh_pts, k=1)[1]
        return vec_particles[nn_idx]

    dists, idxs = tree.query(mesh_pts, k=k)  # (V,k)
    w = 1.0 / (dists + 1e-12)
    w /= w.sum(axis=1, keepdims=True)
    return (vec_particles[idxs] * w[..., None]).sum(axis=1)

def signed_normal_displacement(mesh, disp_vec_mesh):
    """
    Returns a copy of mesh with normals computed, and signed displacement (V,)
    signed = disp · n
    """
    m = mesh.copy()
    m.compute_normals(point_normals=True, cell_normals=False, inplace=True,
                      auto_orient_normals=True, consistent_normals=True)
    n = m.point_normals
    signed = np.einsum("ij,ij->i", disp_vec_mesh, n)
    return m, signed

def save_colorbar_png(
    out_path,
    cmap="PuOr",
    vmin=-1.0,
    vmax=1.0,
    label="Signed normal displacement from mean",
    horizontal=True,
    n_ticks=5,
    fmt="%.2f",
    dpi=300,
    transparent=True
):
    cmap_obj = plt.get_cmap(cmap)
    norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)

    fig = plt.figure(figsize=(6, 1.2) if horizontal else (1.2, 6))
    ax = fig.add_axes([0.05, 0.35, 0.90, 0.35] if horizontal else [0.35, 0.05, 0.35, 0.90])

    cb = mpl.colorbar.ColorbarBase(
        ax,
        cmap=cmap_obj,
        norm=norm,
        orientation="horizontal" if horizontal else "vertical",
        ticks=np.linspace(vmin, vmax, n_ticks),
        format=mpl.ticker.FormatStrFormatter(fmt),
    )
    cb.set_label(label)

    fig.savefig(out_path, dpi=dpi, transparent=transparent, bbox_inches="tight", pad_inches=0.02)
    plt.close(fig)
    

def visualize_mode_strip(mode_idx):
    mode_vec = pca_modes[:, mode_idx].reshape(-1, 3)
    sd = float(np.sqrt(eigenvalues[mode_idx]))

    # 1 SD displacement field at particles -> mesh
    disp_particles_1sd = sd * mode_vec
    disp_mesh_1sd = map_vectors_particles_to_mesh(disp_particles_1sd, k=kNN)  # (V,3)

    # Compute mean mesh normals ONCE and keep them fixed
    mean_with_normals = mean_mesh.copy()
    mean_with_normals.compute_normals(
        point_normals=True,
        cell_normals=False,
        inplace=True,
        auto_orient_normals=True,
        consistent_normals=True
    )
    normals = mean_with_normals.point_normals  # (V,3)

    # Signed normal displacement for 1 SD (scalar per vertex)
    signed_1sd = np.einsum("ij,ij->i", disp_mesh_1sd, normals)  # (V,)

    # SD steps you want
    steps = [-3.0, -1.5, 0.0, 1.5, 3.0]
    sd_labels = ["-3", "-1.5", "mean", "+1.5", "+3"]

    # Shared color limits across all 5 panels (based on max step)
    max_step = max(abs(s) for s in steps)
    clim = (float(np.max(np.abs(signed_1sd)) * max_step) + 1e-12)

    p = pv.Plotter(
        off_screen=True,
        window_size=(3000, 1200),
        shape=(1, 5),
        border=False,   # removes vertical separators
    )

    cam_pos = get_muscle_camera_position(mean_mesh, view="cross2", flip=False)

    # --- draw the 5 meshes ---
    for j, s in enumerate(steps):
        p.subplot(0, j)

        if s == 0.0:
            mesh_s = mean_mesh.copy()
        else:
            mesh_s = mean_mesh.copy()
            mesh_s.points = mean_mesh.points + s * disp_mesh_1sd

        # Heatmap scalar: displacement from mean along normals (same reference normals)
        mesh_s["signed_disp"] = s * signed_1sd  # (V,)

        p.add_mesh(
            mesh_s,
            scalars="signed_disp",
            cmap=cmap,
            clim=[-clim, clim],          # SAME scale for all 5
            smooth_shading=True,
            show_edges=False,
            scalar_bar_args=None,         # prevent auto bar
            show_scalar_bar=False,        # <-- extra safety across pyvista versions
        )

        p.camera_position = cam_pos

        # ---- SD tick (local to this subplot) ----
        if j == 2:
            p.add_text(sd_labels[j] + "\n\n", position="lower_edge", font_size=18)
        else:
            p.add_text(sd_labels[j] + "\n", position="lower_edge", font_size=18)



    # --- GLOBAL TEXT (add once, AFTER the loop) ---
    # IMPORTANT: set subplot to middle so older pyvista anchors to the full window more reliably
    p.subplot(0, 2)

    # Title at top center with margin
    var_pct = 100 * eigenvalues[mode_idx] / np.sum(eigenvalues)
    p.add_text(
        f"\nPC {mode_idx+1} ({var_pct:.1f}%)",
        position="upper_edge",
        viewport=True,
        font_size=24,
    )

    p.add_text(
        "Standard deviations",
        position="lower_edge",
        viewport=True,
        font_size=18,
    )

    # ONE colorbar for the whole figure (attach it while in any subplot)
    # clim is currently a positive scalar. Your actual limits are [-clim, clim]
    vmin, vmax = -clim, clim

    save_colorbar_png(
        out_path=output_dir / f"pc{mode_idx+1}_colorbar.png",
        cmap=cmap,
        vmin=vmin,
        vmax=vmax,
        label= "Surface normal displacement (mm)",
        horizontal=True,
        n_ticks=5,
        fmt="%.2f",
        dpi=300,
        transparent=True
    )

    p.render()
    p.screenshot(str(output_dir / f"pc{mode_idx+1}_sd_heatmap.png"))
    p.close()

# ----------------------------
# RUN
# ----------------------------
for i in range(num_modes_to_visualize):
    visualize_mode_strip(i)

print("Saved to:", output_dir)

Saved to: projects/semimembranosus/semimembranosus_novolume_left_analysis/pcs_heatmaps


# Bootstrapping

In [ ]:
# =========================
#  DEPENDENCIES
# =========================
import os, numpy as np, pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree
from sklearn.utils import resample
try:
    from tqdm.auto import tqdm
    HAS_TQDM = True
except Exception:
    HAS_TQDM = False

def t_range(n, desc=""):
    if HAS_TQDM: return tqdm(range(n), total=n, desc=desc, unit="run")
    return range(n)

# =========================
#  CONFIG — paths & models
# =========================
muscle_name = "biceps_femoris_long"
side = "left"
models = ["rigid", "novolume"]
extensor = True

# particles directories:
main_dir = Path("projects") / f"{muscle_name}"
particles_dir = {m: main_dir / f"{muscle_name}_{m}_{side}_particles" for m in models}

# Bootstrap params
N_BOOTSTRAP = 50
TRAIN_FRAC = 0.8
SEED = 42
rng = np.random.default_rng(SEED)

# Specificity params
COMPUTE_SPECIFICITY = True
N_SPEC_SAMPLES_PER_M = 80   # forward: synth per mode
N_SPEC_POOL_PER_M    = 160  # backward: synth pool per mode

# =========================
#  UTILITIES 
# =========================
def list_all_particles(pdir: Path):
    """
    Loads only *_local.particles files as Nx3 arrays.
    Skips malformed or unreadable ones automatically.
    """
    # only use files containing '_local'
    files = sorted([f for f in pdir.glob("*_local.particles")])
    if not files:
        raise FileNotFoundError(f"No *_local.particles found in {pdir}")

    good, bad = [], []
    for f in files:
        try:
            arr = np.loadtxt(f)
            if arr.ndim == 2 and arr.shape[1] == 3:
                good.append(arr)
            else:
                bad.append(f.name)
        except Exception:
            bad.append(f.name)

    if bad:
        print(f"⚠️ Skipped {len(bad)} malformed files:")
        print("   ", ", ".join(bad[:5]), "..." if len(bad) > 5 else "")
    print(f"✅ Loaded {len(good)} valid *_local.particles files from {pdir.name}")

    return good, files


def pca_from_shapes(shapes_list):
    X = np.stack([s.flatten() for s in shapes_list])     # S x 3N
    mean_vec = X.mean(axis=0)
    centered = X - mean_vec
    U, S, VT = np.linalg.svd(centered, full_matrices=False)
    eigvals = (S**2) / (len(X) - 1) if len(X) > 1 else np.zeros_like(S)
    modes = VT.T                                        # 3N x M
    scores = centered @ VT.T                            # S x M
    return mean_vec, modes, eigvals, scores

def reconstruct(shape_xyz, mean_vec, modes, n_modes):
    n = min(n_modes, modes.shape[1])
    coeffs = (shape_xyz.flatten() - mean_vec) @ modes[:, :n]
    recon = mean_vec + modes[:, :n] @ coeffs
    return recon.reshape(-1,3)

def msd(pc1, pc2):
    return float(np.mean(np.linalg.norm(pc1-pc2, axis=1)))

def hausdorff_distance(pc1, pc2):
    t1, t2 = cKDTree(pc1), cKDTree(pc2)
    d1, _ = t1.query(pc2); d2, _ = t2.query(pc1)
    return float(max(d1.max(), d2.max()))

def cumulative_modes_at_threshold(eigvals, thresh=0.95):
    ev = np.maximum(np.asarray(eigvals,float),0.0)
    tot = ev.sum() if ev.sum()>0 else 1.0
    cum = np.cumsum(ev)/tot
    return int(np.argmax(cum>=thresh))+1

def sample_random_shape(mean_vec, modes, eigvals, m, rng):
    m = min(m, modes.shape[1], len(eigvals))
    a = rng.normal(0.0, np.sqrt(np.maximum(eigvals[:m],0.0)), size=m)
    vec = mean_vec + modes[:, :m] @ a
    return vec.reshape(-1,3)

def specificity_forward_curve_msd(train_shapes, mean_vec, modes, eigvals, M_run, rng, n_synth=80):
    vals=[]
    for m in range(1, M_run+1):
        dmins=[]
        for _ in range(n_synth):
            syn = sample_random_shape(mean_vec, modes, eigvals, m, rng)
            dmins.append(min(msd(syn, t) for t in train_shapes))
        vals.append(float(np.mean(dmins)))
    return np.array(vals)

def specificity_backward_curve_msd(train_shapes, mean_vec, modes, eigvals, M_run, rng, n_pool=160):
    vals=[]
    for m in range(1, M_run+1):
        pool=[sample_random_shape(mean_vec, modes, eigvals, m, rng) for _ in range(n_pool)]
        dmins=[]
        for t in train_shapes:
            dmins.append(min(msd(t, syn) for syn in pool))
        vals.append(float(np.mean(dmins)))
    return np.array(vals)

# =========================
#  BOOTSTRAP per model
# =========================
def run_bootstrap_for_model(pdir: Path, n_boot=100, train_frac=0.8, rng=None, compute_spec=True):
    shapes, _ = list_all_particles(pdir)
    N = len(shapes)
    if N < 6: raise ValueError(f"Need >=6 shapes, got {N} in {pdir}")

    per_msd_train, per_msd_test = [], []
    per_compactness, per_specF, per_specB = [], [], []
    per_m95 = []

    K_common = None
    for b in t_range(n_boot, f"Bootstrap [{pdir.name}]"):
        # consistent split sizes, new permutation each run
        perm = rng.permutation(N)
        ntr = max(1, int(round(train_frac*N)))
        train_idx, test_idx = perm[:ntr], perm[ntr:]

        train_shapes = [shapes[i] for i in train_idx]
        test_shapes  = [shapes[i] for i in test_idx]

        mean_vec, modes, eigvals, _ = pca_from_shapes(train_shapes)
        if modes.size == 0: continue

        # number of usable modes this run
        M_run = int(np.sum(eigvals > 0)) if np.any(eigvals>0) else 1
        M_run = max(1, min(M_run, modes.shape[1]))
        K_common = M_run if K_common is None else min(K_common, M_run)

        # curves up to M_run
        msd_tr, msd_te = [], []
        for m in range(1, M_run+1):
            msd_tr.append(np.mean([msd(reconstruct(s, mean_vec, modes, m), s) for s in train_shapes]))
            msd_te.append(np.mean([msd(reconstruct(s, mean_vec, modes, m), s) for s in test_shapes]))
        per_msd_train.append(np.array(msd_tr))
        per_msd_test.append(np.array(msd_te))

        # compactness (true cumulative variance of TRAIN eigvals)
        ev = np.maximum(np.asarray(eigvals,float),0.0)
        tot = ev.sum() if ev.sum()>0 else 1.0
        per_compactness.append(np.cumsum(ev[:M_run])/tot)
        m95 = cumulative_modes_at_threshold(eigvals, 0.95)
        per_m95.append(min(m95, M_run))

        if compute_spec:
            per_specF.append(specificity_forward_curve_msd(train_shapes, mean_vec, modes, eigvals, M_run, rng,
                                                           n_synth=N_SPEC_SAMPLES_PER_M))
            per_specB.append(specificity_backward_curve_msd(train_shapes, mean_vec, modes, eigvals, M_run, rng,
                                                            n_pool=N_SPEC_POOL_PER_M))

    if K_common is None: raise RuntimeError("No valid runs produced K_common.")

    # truncate all runs to K_common and stack: (R x K)
    def T(stack_list):
        return np.stack([arr[:K_common] for arr in stack_list], axis=0)

    msd_tr_mat  = T(per_msd_train)
    msd_te_mat  = T(per_msd_test)
    comp_mat    = T(per_compactness)
    specF_mat   = T(per_specF) if per_specF else None
    specB_mat   = T(per_specB) if per_specB else None
    modes_axis  = np.arange(1, K_common+1)

    return {
        "modes": modes_axis,
        "msd_train": msd_tr_mat,
        "msd_test":  msd_te_mat,
        "compactness": comp_mat,
        "specF": specF_mat,
        "specB": specB_mat,
        "m95_list": np.array(per_m95, int)
    }

# Run bootstrap for all three models
boot = {}
for m in models:
    boot[m] = run_bootstrap_for_model(particles_dir[m],
                                      n_boot=N_BOOTSTRAP,
                                      train_frac=TRAIN_FRAC,
                                      rng=rng,
                                      compute_spec=COMPUTE_SPECIFICITY)

# =========================
#  PLOTTING + SAVE + DISPLAY
# =========================
def plot_thin_plus_mean(ax, x, mat, color, label):
    """Helper: many thin runs + one thick mean."""
    for row in mat:
        ax.plot(x, row, color=color, alpha=0.07, linewidth=1)
    ax.plot(x, np.mean(mat, axis=0), color=color, linewidth=3, label=label)


# --- 🔹 Set output folder for all bootstrap figures ---
save_dir = main_dir

# ===================================================
# 1️⃣ COMPACTNESS (with 95% mark)
# ===================================================
fig, axes = plt.subplots(1, 2, figsize=(12,4), sharey=True)
for i, m in enumerate(models):
    x = boot[m]["modes"]
    comp = boot[m]["compactness"]
    plot_thin_plus_mean(axes[i], x, comp, "green", "Compactness (mean)")
    m95_med = int(np.median(boot[m]["m95_list"]))
    axes[i].axvline(m95_med, color="gray", linestyle="--", label=f"95% @ {m95_med}")
    axes[i].set_title(m.upper())
    axes[i].set_xlabel("Number of Modes")
    axes[i].set_ylim(0, 1.0)
    if i == 0:
        axes[i].set_ylabel("Cumulative Variance Explained")
    axes[i].legend()

plt.suptitle(f"{muscle_name.upper()} — Compactness (Bootstrap {N_BOOTSTRAP}×, 80/20)", y=1.02)
plt.tight_layout()

# Save and show
comp_path = save_dir / f"{muscle_name}_compactness_bootstrap.png"
plt.savefig(comp_path, dpi=300, bbox_inches="tight")
print(f" Saved compactness plot: {comp_path}")
plt.show()


# ===================================================
# 2️⃣ MSD (train vs test)
# ===================================================
fig, axes = plt.subplots(1, 2, figsize=(12,4), sharey=True)
for i, m in enumerate(models):
    x = boot[m]["modes"]
    plot_thin_plus_mean(axes[i], x, boot[m]["msd_test"], "red", "Test MSD (mean)")
    plot_thin_plus_mean(axes[i], x, boot[m]["msd_train"], "blue", "Train MSD (mean)")
    axes[i].set_title(m.upper())
    axes[i].set_xlabel("Number of Modes")
    if i == 0:
        axes[i].set_ylabel("MSD (mm)")
    axes[i].legend()

plt.suptitle(f"{muscle_name.upper()} — MSD vs Modes (Bootstrap {N_BOOTSTRAP}×, 80/20)", y=1.02)
plt.tight_layout()

# Save and show
msd_path = save_dir / f"{muscle_name}_msd_bootstrap.png"
plt.savefig(msd_path, dpi=300, bbox_inches="tight")
print(f" Saved MSD plot: {msd_path}")
plt.show()


# ===================================================
# 3️⃣ SPECIFICITY (forward/backward)
# ===================================================
if COMPUTE_SPECIFICITY:
    fig, axes = plt.subplots(1, 2, figsize=(12,4), sharey=True)
    for i, m in enumerate(models):
        x = boot[m]["modes"]
        if boot[m]["specF"] is not None:
            plot_thin_plus_mean(axes[i], x, boot[m]["specF"], "orange", "Specificity Forward (mean)")
        if boot[m]["specB"] is not None:
            plot_thin_plus_mean(axes[i], x, boot[m]["specB"], "purple", "Specificity Backward (mean)")
        axes[i].set_title(m.upper())
        axes[i].set_xlabel("Number of Modes")
        if i == 0:
            axes[i].set_ylabel("Mean min-MSD (mm)")
        axes[i].legend()

    plt.suptitle(f"{muscle_name.upper()} — Specificity vs Modes (Bootstrap {N_BOOTSTRAP}×, 80/20)", y=1.02)
    plt.tight_layout()

    # Save and show
    spec_path = save_dir / f"{muscle_name}_specificity_bootstrap.png"
    plt.savefig(spec_path, dpi=300, bbox_inches="tight")
    print(f"💾 Saved Specificity plot: {spec_path}")
    plt.show()

# ===================================================
#  PRINT SUMMARY @ PC=3 (mean ± SD across bootstraps)
# ===================================================
PC = 3

def mean_sd(x):
    x = np.asarray(x, float)
    return float(np.mean(x)), float(np.std(x, ddof=1)) if x.size > 1 else (float(np.mean(x)), 0.0)

rows = []
for m in models:
    K = boot[m]["modes"].size
    if K < PC:
        print(f"⚠️ {m}: only {K} common modes available, cannot report PC{PC}.")
        continue

    idx = PC - 1

    comp_pc = boot[m]["compactness"][:, idx]
    gen_pc  = boot[m]["msd_test"][:, idx]

    comp_mean, comp_sd = mean_sd(comp_pc)
    gen_mean, gen_sd   = mean_sd(gen_pc)

    if boot[m]["specF"] is not None:
        specF_mean, specF_sd = mean_sd(boot[m]["specF"][:, idx])
    else:
        specF_mean, specF_sd = (np.nan, np.nan)

    if boot[m]["specB"] is not None:
        specB_mean, specB_sd = mean_sd(boot[m]["specB"][:, idx])
    else:
        specB_mean, specB_sd = (np.nan, np.nan)

    rows.append({
        "model": m,
        "compactness@PC3_mean": comp_mean,
        "compactness@PC3_SD": comp_sd,
        "generalizability_testMSD@PC3_mean_mm": gen_mean,
        "generalizability_testMSD@PC3_SD_mm": gen_sd,
        "specificity_forward@PC3_mean_mm": specF_mean,
        "specificity_forward@PC3_SD_mm": specF_sd,
        "specificity_backward@PC3_mean_mm": specB_mean,
        "specificity_backward@PC3_SD_mm": specB_sd,
        "n_boot_runs": boot[m]["msd_test"].shape[0],
    })

summary_df = pd.DataFrame(rows)

print("\n===== Bootstrap summary at PC3 =====")
print(summary_df.to_string(index=False))

summary_df

✅ Loaded 101 valid *_local.particles files from biceps_femoris_long_rigid_left_particles


Bootstrap [biceps_femoris_long_rigid_left_particles]:   0%|          | 0/50 [00:00<?, ?run/s]

KeyboardInterrupt: 

# Regression

## Single Muscle

In [24]:
import pandas as pd
import numpy as np
from pathlib import Path
import statsmodels.formula.api as smf
import re

# ============================================================
# CONFIG — SINGLE MUSCLE
# ============================================================
muscle_name = "semimembranosus"   # CHANGE as needed
IS_FLEXOR   = False               # False extensor, True flexor
side = "left"

modifiers = ["rigid", "novolume"]
N_PCS = 3

chars_path = Path("MOTION_characteristics.xlsx")
project_dir = Path("projects")

# ============================================================
# FORCE COLUMN (ISOM ONLY)
# ============================================================
outcome_col = "avpt_isom_k_f" if IS_FLEXOR else "avpt_isom_k_e"

# ============================================================
# LOAD CHARACTERISTICS (only need outcome here)
# ============================================================
chars_raw = pd.read_excel(chars_path)
chars_raw.columns = [c.lower().strip() for c in chars_raw.columns]
if "participant id" not in chars_raw.columns:
    raise ValueError("Expected column 'participant id' in MOTION_characteristics.xlsx")

chars = chars_raw[["participant id", outcome_col]].copy()
chars = chars.rename(columns={"participant id": "participant_id"})
chars["participant_id"] = chars["participant_id"].astype(str).str.upper()

chars[outcome_col] = pd.to_numeric(chars[outcome_col], errors="coerce")
chars = chars.dropna(subset=[outcome_col])
chars = chars[chars[outcome_col] >= 0].copy()
chars = chars.drop_duplicates(subset=["participant_id"])

# ============================================================
# LOAD ORIGINAL VOLUMES (LEFT SIDE ONLY)
# ============================================================
volpath = Path(f"data/{muscle_name}/{muscle_name}_original_volumes.csv")
vol_df = pd.read_csv(volpath)

if "shape_name" not in vol_df.columns or "original_volume_mm3" not in vol_df.columns:
    raise ValueError(f"{volpath} must contain shape_name and original_volume_mm3")

vol_df = vol_df[vol_df["shape_name"].str.contains("_left", case=False, na=False)].copy()

def extract_left_id(name):
    m = re.search(r"(?i)(motion\d+)_left", str(name))
    return m.group(1).upper() if m else None

vol_df["participant_id"] = vol_df["shape_name"].apply(extract_left_id)
vol_df = vol_df.dropna(subset=["participant_id"]).copy()

vol_df["original_volume_mm3"] = pd.to_numeric(vol_df["original_volume_mm3"], errors="coerce")
vol_df = vol_df.dropna(subset=["original_volume_mm3"]).copy()

# if duplicates exist, average per subject
vol_df = vol_df.groupby("participant_id", as_index=False)["original_volume_mm3"].mean()

# z-score volume
vol_df["volume_z"] = (
    vol_df["original_volume_mm3"] - vol_df["original_volume_mm3"].mean()
) / vol_df["original_volume_mm3"].std(ddof=0)

vol_df = vol_df[["participant_id", "volume_z"]]

# ============================================================
# LOAD PC SCORES
# ============================================================
def load_pc_single(muscle, modifier):
    path = (
        project_dir /
        muscle /
        f"{muscle}_{modifier}_{side}_analysis" /
        "scores_all_standardized.csv"
    )

    df = pd.read_csv(path)
    df.columns = [c.lower().strip() for c in df.columns]

    # normalize ID column to participant_id
    for cand in ["participant_id", "participant id", "id", "subjectid", "subject_id"]:
        if cand in df.columns:
            df = df.rename(columns={cand: "participant_id"})
            break
    if "participant_id" not in df.columns:
        raise ValueError(f"No subject ID column found in {path}")

    df["participant_id"] = df["participant_id"].astype(str).str.upper()

    pc_cols = sorted([c for c in df.columns if c.startswith("pc")],
                     key=lambda x: int(x.replace("pc", "")))[:N_PCS]

    out = df[["participant_id"] + pc_cols].copy()
    for c in pc_cols:
        out[c] = pd.to_numeric(out[c], errors="coerce")
    out = out.dropna(subset=pc_cols)

    # ensure names are pc1..pcN
    rename_map = {pc_cols[i]: f"pc{i+1}" for i in range(len(pc_cols))}
    out = out.rename(columns=rename_map)
    return out

# ============================================================
# TABLE BUILDERS
# ============================================================
def mixedlm_tables_with_fitinfo(res):
    params = res.params
    bse = res.bse
    zval = params / bse
    pvals = res.pvalues
    ci = res.conf_int()
    ci.columns = ["ci_low", "ci_high"]

    coef_df = pd.concat([params, bse, zval, pvals, ci], axis=1)
    coef_df.columns = ["coef", "std_err", "z", "p_value", "ci_low", "ci_high"]
    coef_df = coef_df.reset_index().rename(columns={"index": "term"})

    fit_info = pd.DataFrame([
        {"term": "AIC",       "coef": getattr(res, "aic", np.nan)},
        {"term": "BIC",       "coef": getattr(res, "bic", np.nan)},
        {"term": "logLik",    "coef": getattr(res, "llf", np.nan)},
        {"term": "n_obs",     "coef": getattr(res, "nobs", np.nan)},
        {"term": "k_params",  "coef": len(getattr(res, "params", []))},
        {"term": "converged", "coef": int(bool(getattr(res, "converged", False)))},
        {"term": "reml",      "coef": 0},  # ML fit
    ])
    for c in ["std_err", "z", "p_value", "ci_low", "ci_high"]:
        fit_info[c] = np.nan

    return pd.concat([coef_df, fit_info], ignore_index=True)

# ============================================================
# OUTPUT FILE
# ============================================================
output_dir = Path("regression")
output_dir.mkdir(parents=True, exist_ok=True)
file_path = output_dir / f"{muscle_name}_mixedlm_PC{N_PCS}.xlsx"

# ============================================================
# RUN MODELS: PC-only + VOLUME-only + PC+volume
# IMPORTANT: reml=False so AIC/BIC exist
# ============================================================
with pd.ExcelWriter(file_path, engine="openpyxl", mode="w") as writer:

    for modifier in modifiers:
        print(f"\n--- Modifier: {modifier} ---")

        pc_df = load_pc_single(muscle_name, modifier)

        df = (
            pc_df
            .merge(chars, on="participant_id", how="inner")
            .merge(vol_df, on="participant_id", how="inner")
        )

        df[outcome_col] = pd.to_numeric(df[outcome_col], errors="coerce")
        df = df.dropna(subset=[outcome_col])
        df = df[df[outcome_col] >= 0].copy()

        pcs = [f"pc{i}" for i in range(1, N_PCS + 1)]

        # -------------------------
        # MODEL 1: PC ONLY
        # -------------------------
        formula1 = f"{outcome_col} ~ " + " + ".join(pcs)
        sheet1 = f"{modifier}_pc_only"
        data1 = f"{sheet1}_DATA"

        try:
            res1 = smf.mixedlm(formula1, df, groups=df["participant_id"], re_formula="1").fit(
                method="lbfgs", maxiter=500, reml=False
            )
            mixedlm_tables_with_fitinfo(res1).to_excel(writer, index=False, sheet_name=sheet1)
            # df[["participant_id", outcome_col] + pcs].to_excel(writer, index=False, sheet_name=data1)
        except Exception as e:
            pd.DataFrame({"error": [str(e)], "formula": [formula1]}).to_excel(writer, index=False, sheet_name=sheet1)

        # -------------------------
        # MODEL 2: PC + VOLUME
        # -------------------------
        formula2 = f"{outcome_col} ~ " + " + ".join(pcs + ["volume_z"])
        sheet2 = f"{modifier}_pc+volume"
        data2 = f"{sheet2}_DATA"

        try:
            res2 = smf.mixedlm(formula2, df, groups=df["participant_id"], re_formula="1").fit(
                method="lbfgs", maxiter=500, reml=False
            )
            mixedlm_tables_with_fitinfo(res2).to_excel(writer, index=False, sheet_name=sheet2)
            # df[["participant_id", outcome_col] + pcs + ["volume_z"]].to_excel(writer, index=False, sheet_name=data2)
        except Exception as e:
            pd.DataFrame({"error": [str(e)], "formula": [formula2]}).to_excel(writer, index=False, sheet_name=sheet2)
    
    # -------------------------
    # MODEL 3: VOLUME ONLY
    # -------------------------
    formula3 = f"{outcome_col} ~ volume_z"
    sheet3 = f"{modifier}_volume"
    data3 = f"{sheet2}_DATA"

    try:
        res2 = smf.mixedlm(formula2, df, groups=df["participant_id"], re_formula="1").fit(
            method="lbfgs", maxiter=500, reml=False
        )
        mixedlm_tables_with_fitinfo(res3).to_excel(writer, index=False, sheet_name=sheet3)
        # df[["participant_id", outcome_col, "volume_z"]].to_excel(writer, index=False, sheet_name=data3)
    except Exception as e:
        pd.DataFrame({"error": [str(e)], "formula": [formula3]}).to_excel(writer, index=False, sheet_name=sheet3)

print("\n✔ Saved:", file_path)



--- Modifier: rigid ---

--- Modifier: novolume ---

✔ Saved: regression/semimembranosus_mixedlm_PC3.xlsx


/Users/fadiahanifa/miniconda3/envs/shapeworks/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)
/Users/fadiahanifa/miniconda3/envs/shapeworks/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)
/Users/fadiahanifa/miniconda3/envs/shapeworks/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)
/Users/fadiahanifa/miniconda3/envs/shapeworks/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not 

## Multi Muscle

In [29]:
import pandas as pd
import numpy as np
from pathlib import Path
import statsmodels.formula.api as smf
import re

# ============================================================
# CONFIG
# ============================================================
project_dir = Path("projects")
group_is_flexor = True
side = "left"
modifiers = ["rigid", "novolume"]
N_PCS = 3

chars_path = Path("MOTION_characteristics.xlsx")
group_dir = "flexor_group_left" if group_is_flexor else "extensor_group_left"

if group_is_flexor:
    muscle_list = ["biceps_femoris_long", "biceps_femoris_short", "semitendinosus", "semimembranosus"]
    outcome_col = "avpt_isom_k_f"
else:
    muscle_list = ["vastus_lateralis", "vastus_medialis", "vastus_intermedius", "rectus_femoris"]
    outcome_col = "avpt_isom_k_e"
output_dir = Path("regression")

print("Muscles in group:", muscle_list)

# ============================================================
# HELPERS
# ============================================================
def safe_zscore(x: pd.Series) -> pd.Series:
    x = pd.to_numeric(x, errors="coerce")
    mu = x.mean()
    sd = x.std(ddof=0)
    if not np.isfinite(sd) or sd == 0:
        return pd.Series(np.zeros(len(x)), index=x.index)
    return (x - mu) / sd

def coerce_and_filter(df: pd.DataFrame, cols, name="df"):
    df = df.copy()
    for c in cols:
        df[c] = pd.to_numeric(df[c], errors="coerce")
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    before = len(df)
    df = df.dropna(subset=cols)
    after = len(df)
    print(f"[CLEAN] {name}: before={before} after_dropna_inf={after}")
    if after == 0:
        raise ValueError(f"{name} became empty after cleaning on columns: {cols}")
    return df

def coef_table(res):
    ci = res.conf_int()
    ci.columns = ["ci_low", "ci_high"]
    out = pd.concat(
        [res.params.rename("coef"),
         res.bse.rename("std_err"),
         res.pvalues.rename("p"),
         ci],
        axis=1
    ).reset_index().rename(columns={"index": "term"})
    return out

def model_stats_row(res, model_name, modifier, formula):
    # MixedLM AIC/BIC are only defined for ML fits (reml=False)
    return {
        "model": model_name,
        "modifier": modifier,
        "formula": formula,
        "n_obs": int(res.nobs),
        "k_params": int(len(res.params)),
        "logLik": float(res.llf),
        "AIC": float(res.aic),
        "BIC": float(res.bic),
        "converged": bool(getattr(res, "converged", False)),
        "reml": False,
    }

# ============================================================
# SUBJECT-LEVEL OUTCOME (TORQUE)
# ============================================================
chars = pd.read_excel(chars_path)
chars.columns = [c.lower().strip() for c in chars.columns]
chars = chars.rename(columns={"participant id": "participant_id"})
chars["participant_id"] = chars["participant_id"].astype(str).str.upper()

if outcome_col not in chars.columns:
    raise ValueError(f"{outcome_col} not found in MOTION_characteristics.xlsx")

chars = chars[["participant_id", outcome_col]].copy()
chars = coerce_and_filter(chars, [outcome_col], name="chars_outcome")
chars = chars[chars[outcome_col] >= 0].copy()
chars = chars.drop_duplicates(subset=["participant_id"])

# ============================================================
# MUSCLE VOLUME PER MUSCLE (LEFT ONLY) -> wide columns: <muscle>_vol_z
# NOTE: do NOT drop rows here (outer merge); complete-case later.
# ============================================================
vol_wide = None
for muscle in muscle_list:
    vdf = pd.read_csv(f"data/{muscle}/{muscle}_original_volumes.csv")
    if "shape_name" not in vdf.columns or "original_volume_mm3" not in vdf.columns:
        raise ValueError(f"{muscle}_original_volumes.csv must contain shape_name and original_volume_mm3")

    vdf = vdf[vdf["shape_name"].str.contains(f"_{side}", case=False, na=False)].copy()
    vdf["participant_id"] = vdf["shape_name"].str.extract(r"(MOTION\d+)")[0].astype(str).str.upper()
    vdf = vdf.dropna(subset=["participant_id"]).copy()

    vdf["original_volume_mm3"] = pd.to_numeric(vdf["original_volume_mm3"], errors="coerce")
    vdf = vdf.dropna(subset=["original_volume_mm3"]).copy()

    vdf = vdf.groupby("participant_id", as_index=False)["original_volume_mm3"].mean()
    vdf[f"{muscle}_vol_z"] = safe_zscore(vdf["original_volume_mm3"])
    vdf = vdf[["participant_id", f"{muscle}_vol_z"]]

    vol_wide = vdf if vol_wide is None else vol_wide.merge(vdf, on="participant_id", how="outer")

vol_terms = [f"{m}_vol_z" for m in muscle_list]
for c in vol_terms:
    vol_wide[c] = pd.to_numeric(vol_wide[c], errors="coerce")

print("[INFO] vol_wide shape:", vol_wide.shape)
print("[INFO] non-missing per volume column:\n", vol_wide[vol_terms].notna().sum())

# ============================================================
# LOAD PCs — WIDE PER MUSCLE
# ============================================================
def load_muscle_pcs(muscle, modifier):
    path = (
        project_dir / muscle /
        f"{muscle}_{modifier}_{side}_analysis" /
        "scores_all_standardized.csv"
    )
    df = pd.read_csv(path)
    df.columns = [c.lower().strip() for c in df.columns]

    if "subjectid" in df.columns:
        df = df.rename(columns={"subjectid": "participant_id"})
    elif "participant id" in df.columns:
        df = df.rename(columns={"participant id": "participant_id"})
    elif "participant_id" not in df.columns:
        raise ValueError(f"No subject ID column in {path}")

    df["participant_id"] = df["participant_id"].astype(str).str.upper()

    pcs = [f"pc{i}" for i in range(1, N_PCS + 1)]
    missing = [p for p in pcs if p not in df.columns]
    if missing:
        raise ValueError(f"Missing PCs {missing} in {path}")

    out = df[["participant_id"] + pcs].copy()
    out = coerce_and_filter(out, pcs, name=f"{muscle}_{modifier}_pcs")

    out = out.rename(columns={
        "pc1": f"{muscle}_pc1",
        "pc2": f"{muscle}_pc2",
        "pc3": f"{muscle}_pc3",
    })
    return out

# ============================================================
# BUILD SUBJECT-LEVEL TABLE
# ============================================================
def build_subject_table(modifier):
    dfs = [load_muscle_pcs(m, modifier) for m in muscle_list]

    subj_df = dfs[0]
    for d in dfs[1:]:
        subj_df = subj_df.merge(d, on="participant_id", how="inner")

    subj_df = (
        subj_df
        .merge(chars, on="participant_id", how="inner")
        .merge(vol_wide, on="participant_id", how="left")
    )

    pc_terms = []
    for m in muscle_list:
        pc_terms += [f"{m}_pc1", f"{m}_pc2", f"{m}_pc3"]

    vol_terms_local = [f"{m}_vol_z" for m in muscle_list]

    # complete-case for the *volume model* will happen later;
    # for now we return full table and lists
    print(f"[INFO] {modifier} pre-drop shape:", subj_df.shape)
    return subj_df, pc_terms, vol_terms_local

# ============================================================
# FIT MIXED MODEL (ML so AIC/BIC exist)
# ============================================================
def fit_model(df, formula):
    model = smf.mixedlm(formula, df, groups=df["participant_id"], re_formula="1")
    return model.fit(method="lbfgs", maxiter=800, reml=False)  # <-- ML!

# ============================================================
# RUN MODELS
# ============================================================
all_stats = []
output_path = output_dir / f"{group_dir}_mixedlm_PC{N_PCS}.xlsx"

with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
    # always keep at least one visible sheet
    for mod in modifiers:
        print(f"\n=== Modifier: {mod} ===")
        subj_df, pc_terms, vol_terms_local = build_subject_table(mod)

        # -------------------------
        # MODEL 1: PC ONLY
        # -------------------------
        need_cols_1 = ["participant_id", outcome_col] + pc_terms
        df1 = subj_df[need_cols_1].copy()
        df1 = coerce_and_filter(df1, [outcome_col] + pc_terms, name=f"{mod}_pc_only_df")
        df1 = df1[df1[outcome_col] >= 0].copy()

        f1 = f"{outcome_col} ~ " + " + ".join(pc_terms)
        print("[FORMULA pc_only]", f1)

        try:
            res1 = fit_model(df1, f1)
            c1 = coef_table(res1)
            # append fit info to coef sheet
            fit_info1 = pd.DataFrame([
                {"term": "AIC", "coef": res1.aic, "std_err": np.nan, "p": np.nan, "ci_low": np.nan, "ci_high": np.nan},
                {"term": "BIC", "coef": res1.bic, "std_err": np.nan, "p": np.nan, "ci_low": np.nan, "ci_high": np.nan},
                {"term": "logLik", "coef": res1.llf, "std_err": np.nan, "p": np.nan, "ci_low": np.nan, "ci_high": np.nan},
                {"term": "n_obs", "coef": res1.nobs, "std_err": np.nan, "p": np.nan, "ci_low": np.nan, "ci_high": np.nan},
                {"term": "converged", "coef": int(res1.converged), "std_err": np.nan, "p": np.nan, "ci_low": np.nan, "ci_high": np.nan},
            ])
            pd.concat([c1, fit_info1], ignore_index=True).to_excel(writer, sheet_name=f"{mod}_pc_only", index=False)

            # data + fitted + resid
            d1 = df1.copy()
            d1["fitted"] = np.asarray(res1.fittedvalues)
            d1["resid"] = np.asarray(res1.resid)
            d1["std_resid"] = d1["resid"] / np.sqrt(res1.scale) if np.isfinite(res1.scale) and res1.scale > 0 else np.nan
            # d1.to_excel(writer, sheet_name=f"{mod}_pc_only_DATA", index=False)

            all_stats.append(model_stats_row(res1, "pc_only", mod, f1))

        except Exception as e:
            pd.DataFrame([{"modifier": mod, "model": "pc_only", "error": str(e), "formula": f1}]).to_excel(
                writer, sheet_name=f"{mod}_pc_only_ERR"[:31], index=False
            )

        # -------------------------
        # MODEL 2: PC + PER-MUSCLE VOLUME
        # -------------------------
        need_cols_2 = ["participant_id", outcome_col] + pc_terms + vol_terms_local
        df2 = subj_df[need_cols_2].copy()
        df2 = coerce_and_filter(df2, [outcome_col] + pc_terms + vol_terms_local, name=f"{mod}_pc_musclevol_df")
        df2 = df2[df2[outcome_col] >= 0].copy()

        f2 = f"{outcome_col} ~ " + " + ".join(pc_terms + vol_terms_local)
        print("[FORMULA pc_musclevol]", f2)

        try:
            res2 = fit_model(df2, f2)
            c2 = coef_table(res2)
            fit_info2 = pd.DataFrame([
                {"term": "AIC", "coef": res2.aic, "std_err": np.nan, "p": np.nan, "ci_low": np.nan, "ci_high": np.nan},
                {"term": "BIC", "coef": res2.bic, "std_err": np.nan, "p": np.nan, "ci_low": np.nan, "ci_high": np.nan},
                {"term": "logLik", "coef": res2.llf, "std_err": np.nan, "p": np.nan, "ci_low": np.nan, "ci_high": np.nan},
                {"term": "n_obs", "coef": res2.nobs, "std_err": np.nan, "p": np.nan, "ci_low": np.nan, "ci_high": np.nan},
                {"term": "converged", "coef": int(res2.converged), "std_err": np.nan, "p": np.nan, "ci_low": np.nan, "ci_high": np.nan},
            ])
            pd.concat([c2, fit_info2], ignore_index=True).to_excel(writer, sheet_name=f"{mod}_pc_musclevol", index=False)

            d2 = df2.copy()
            d2["fitted"] = np.asarray(res2.fittedvalues)
            d2["resid"] = np.asarray(res2.resid)
            d2["std_resid"] = d2["resid"] / np.sqrt(res2.scale) if np.isfinite(res2.scale) and res2.scale > 0 else np.nan
            #d2.to_excel(writer, sheet_name=f"{mod}_pc_musclevol_DATA", index=False)

            all_stats.append(model_stats_row(res2, "pc+volume", mod, f2))

        except Exception as e:
            pd.DataFrame([{"modifier": mod, "model": "pc+volume", "error": str(e), "formula": f2}]).to_excel(
                writer, sheet_name=f"{mod}_pc_musclevol_ERR"[:31], index=False
            )
print("\n✔ Saved:", output_path)


Muscles in group: ['biceps_femoris_long', 'biceps_femoris_short', 'semitendinosus', 'semimembranosus']
[CLEAN] chars_outcome: before=126 after_dropna_inf=115
[INFO] vol_wide shape: (101, 5)
[INFO] non-missing per volume column:
 biceps_femoris_long_vol_z     101
biceps_femoris_short_vol_z    101
semitendinosus_vol_z          101
semimembranosus_vol_z         101
dtype: int64

=== Modifier: rigid ===
[CLEAN] biceps_femoris_long_rigid_pcs: before=101 after_dropna_inf=101
[CLEAN] biceps_femoris_short_rigid_pcs: before=101 after_dropna_inf=101
[CLEAN] semitendinosus_rigid_pcs: before=101 after_dropna_inf=101
[CLEAN] semimembranosus_rigid_pcs: before=101 after_dropna_inf=101
[INFO] rigid pre-drop shape: (97, 18)
[CLEAN] rigid_pc_only_df: before=97 after_dropna_inf=97
[FORMULA pc_only] avpt_isom_k_f ~ biceps_femoris_long_pc1 + biceps_femoris_long_pc2 + biceps_femoris_long_pc3 + biceps_femoris_short_pc1 + biceps_femoris_short_pc2 + biceps_femoris_short_pc3 + semitendinosus_pc1 + semitendinosu

/Users/fadiahanifa/miniconda3/envs/shapeworks/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)
/Users/fadiahanifa/miniconda3/envs/shapeworks/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)



=== Modifier: novolume ===
[CLEAN] biceps_femoris_long_novolume_pcs: before=101 after_dropna_inf=101
[CLEAN] biceps_femoris_short_novolume_pcs: before=101 after_dropna_inf=101
[CLEAN] semitendinosus_novolume_pcs: before=101 after_dropna_inf=101
[CLEAN] semimembranosus_novolume_pcs: before=101 after_dropna_inf=101
[INFO] novolume pre-drop shape: (97, 18)
[CLEAN] novolume_pc_only_df: before=97 after_dropna_inf=97
[FORMULA pc_only] avpt_isom_k_f ~ biceps_femoris_long_pc1 + biceps_femoris_long_pc2 + biceps_femoris_long_pc3 + biceps_femoris_short_pc1 + biceps_femoris_short_pc2 + biceps_femoris_short_pc3 + semitendinosus_pc1 + semitendinosus_pc2 + semitendinosus_pc3 + semimembranosus_pc1 + semimembranosus_pc2 + semimembranosus_pc3
[CLEAN] novolume_pc_musclevol_df: before=97 after_dropna_inf=97
[FORMULA pc_musclevol] avpt_isom_k_f ~ biceps_femoris_long_pc1 + biceps_femoris_long_pc2 + biceps_femoris_long_pc3 + biceps_femoris_short_pc1 + biceps_femoris_short_pc2 + biceps_femoris_short_pc3 + se

/Users/fadiahanifa/miniconda3/envs/shapeworks/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)
/Users/fadiahanifa/miniconda3/envs/shapeworks/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)



✔ Saved: regression/flexor_group_left_mixedlm_PC3.xlsx
